# An Array of Sequences

## Overview

이 장에서 다룰 것들은 시퀀스. 즉, 순서대로 물건을 담을수 있는 자료구조를 다룹니다.

그 자료구조들 중, "여러가지를 담을수 있느냐", "한 종류만 다룰수 있느냐"를 가지고 나누면 이렇게 됩니다.

| Type | Sequences |
|------|-----------|
| Container | list, tuple, collections.deque |
| Flat | str, bytes, bytearray, memoryview, array.array |

다른 기준으로는, "담은 내용물을 나중에 바꿀수 있느냐"를 기준으로 나누면 이렇게 됩니다.

| Type | Sequences |
|------|-----------|
| Mutable | list, bytearray, array.array, collections.deque, memoryview |
| Immutable | tuple, str, bytes |

## List Comprehension

1장에서 배운 Pythonic ([built-in function](https://docs.python.org/3/library/functions.html)으로 편리하게 만드는 것)과 함께 파이썬을 강력하게 만드는 다른 한 축이 List Comprehension 이라고 생각합니다.

sequence 관련 연산을 one-liner 로 많이 만들여주죠. 예제들을 봅시다.

In [1]:
# List Comprehension 사용하지 않음
symbols = '!@#$%^&*'
codes = []
for symbol in symbols:
    codes.append(ord(symbol))
codes

[33, 64, 35, 36, 37, 94, 38, 42]

In [2]:
# List Comprehension 사용
symbols = '!@#$%^&*'
codes = [ord(symbol) for symbol in symbols]
codes

[33, 64, 35, 36, 37, 94, 38, 42]

`map()`, `filter()`를 사용해도 같은 효과를 냅니다.

In [3]:
codes = list(map(ord, symbols))
codes

[33, 64, 35, 36, 37, 94, 38, 42]

단, `map()`, `filter()`를 사용하는것이 더 빠르지는 않다고하네요. 실험해볼까요?

In [4]:
# https://github.com/fluentpython/example-code/blob/master/02-array-seq/listcomp_speed.py

import timeit

TIMES = 10000

SETUP = """
symbols = '$¢£¥€¤'
def non_ascii(c):
    return c > 127
"""

def clock(label, cmd):
    res = timeit.repeat(cmd, setup=SETUP, number=TIMES)
    print(label, *(f'{x:.3f}' for x in res))

clock('listcomp        :', '[ord(s) for s in symbols if ord(s) > 127]')
clock('listcomp + func :', '[ord(s) for s in symbols if non_ascii(ord(s))]')
clock('filter + lambda :', 'list(filter(lambda c: c > 127, map(ord, symbols)))')
clock('filter + func   :', 'list(filter(non_ascii, map(ord, symbols)))')

listcomp        : 0.012 0.013 0.011 0.009 0.010
listcomp + func : 0.014 0.017 0.014 0.014 0.013
filter + lambda : 0.012 0.012 0.012 0.012 0.012
filter + func   : 0.011 0.011 0.011 0.011 0.012


아주 약간 빠르기는 빠르네요. 하지만, 성능의 차이가 있다고 할 만큼은 아닌 것 같습니다.

다음으로, cartesian product 를 해야 하는 경우를 살펴보겠습니다. 다른 color, size 의 t-shirt 쌍을 만든다고 해봅시다.

    (black, S)
    (black, M)
    (black, L)
    (white, S)
    (white, M)
    (white, L)
    
간단히 이렇게 할 수 있겠죠.

In [5]:
colors = ['black', 'white']
sizes = ['S', 'M', 'L']
tshirts = []
for color in colors:
    for size in sizes:
        tshirts.append((color, size))
tshirts

[('black', 'S'),
 ('black', 'M'),
 ('black', 'L'),
 ('white', 'S'),
 ('white', 'M'),
 ('white', 'L')]

`itertools`라는 좋은 패키기가 있습니다.

In [6]:
import itertools
tshirts = []
for color, size in itertools.product(colors, sizes):
    tshirts.append((color, size))
tshirts

[('black', 'S'),
 ('black', 'M'),
 ('black', 'L'),
 ('white', 'S'),
 ('white', 'M'),
 ('white', 'L')]

하지만, 가장 읽기 쉬운것은 List Comprehension 입니다!

In [7]:
tshirts = [(color, size) for color in colors for size in sizes]
tshirts

[('black', 'S'),
 ('black', 'M'),
 ('black', 'L'),
 ('white', 'S'),
 ('white', 'M'),
 ('white', 'L')]

## Tuple and Immutability
    
튜플은 기본적으로 *immutable* 자료구조입니다. 책에 그에 관해 여러 예제를 주니 그냥 가볍게 읽어보시면 되겠습니다.

왜 튜플 섹션에 있는지 모르겠지만, `*`를 사용한 unpacking 은 재밌습니다.

In [8]:
a, b, *rest = range(5)
print(a, b, rest)
a, b, *rest = range(3)
print(a, b, rest)
a, b, *rest = range(2)
print(a, b, rest)

0 1 [2, 3, 4]
0 1 [2]
0 1 []


### NamedTuple

`collections.namedtuple` 은 아주 자주쓰이죠. 힘들게 `class`를 정의 할 필요없이, one-liner로 많이 씁니다.

In [9]:
from collections import namedtuple
City = namedtuple('City', 'name country population coordinates')
tokyo = City('Tokyo', 'JP', 36.933, (35.689722, 139.691667))
print(tokyo)
print(tokyo.population)
print(tokyo.coordinates)

City(name='Tokyo', country='JP', population=36.933, coordinates=(35.689722, 139.691667))
36.933
(35.689722, 139.691667)


## Slicing

파이썬의 슬라이싱은 재미있는게 많습니다. 기본적인 것들 몇개를 드리겠습니다.

In [10]:
l = list(range(10, 70, 10))
print(l)
print(l[:2])
print(l[2:])
print(l[::2])
print(l[::-1])

[10, 20, 30, 40, 50, 60]
[10, 20]
[30, 40, 50, 60]
[10, 30, 50]
[60, 50, 40, 30, 20, 10]


`Numpy`는 multi-dimensional array 의 slicing 도 합니다. 알아두시면 좋죠.

In [11]:
import numpy as np
l = np.array(range(12)).reshape(3, 4)
print(l)
print(l[:, 1])
print(l[1, :])

[[ 0  1  2  3]
 [ 4  5  6  7]
 [ 8  9 10 11]]
[1 5 9]
[4 5 6 7]


`Pandas`의 슬라이싱도 알아두세요.

In [12]:
import pandas as pd
l = np.array(range(12)).reshape(3, 4)
df = pd.DataFrame(l, columns='A B C D'.split())
display(df)
display(df[['B', 'D']])
display(df.loc[1])
display(df.loc[[1, 2]])
display(df.iloc[1])

,A,B,C,D
0,0,1,2,3
1,4,5,6,7
2,8,9,10,11


,B,D
0,1,3
1,5,7
2,9,11


A    4
B    5
C    6
D    7
Name: 1, dtype: int64

,A,B,C,D
1,4,5,6,7
2,8,9,10,11


A    4
B    5
C    6
D    7
Name: 1, dtype: int64

### Example: is_palindrome()

[팔린드롬](https://examples.yourdictionary.com/palindrome-examples.html)이 뭔가요? `Anna`, `Civic`, `Level`처럼 앞으로 읽으나 뒤로 읽으나 같은 단어 혹은 문장을 말합니다.

팔린드롬을 짜면 보통 이런식으로 하죠.

In [13]:
palindromes = ['Anna', 'Civic', 'Kayak', 'Level', 'Madam', 'Mom', 'Noon', 'Racecar', 'Radar',
               'Redder', 'Refer', 'Repaper', 'Rotator', 'Rotor', 'Sagas', 'Solos', 'Stats',
               'Tenet', 'Wow']
def is_palindrome(s):
    for i in range(len(s)):
        if s[i].lower() != s[len(s) - i - 1]:
            return False
    return True
all(is_palindrome(s.lower()) for s in palindromes)

True

one-liner로 하면 이렇게 됩니다:

In [14]:
def is_palindrome(s):
    return all(s[i] == s[len(s)-i-1] for i in range(len(s)))
all(is_palindrome(s.lower()) for s in palindromes)

True

여기서 slicing (python indexing) 의 매직이 있습니다. `~` operator 가 binary one's complement 임을 이용하죠.

즉, `~i` == `-i+1`.

`~0` == `-1`

`~1` == `-2`

`~2` == `-3`

...

이 속성을 이용해서, 이 문제에서 `~`를 쓰면 훨씬 더 효율적이고 멋지게 풀 수 있습니다.

In [15]:
def is_palindrome(s):
    return all(s[i] == s[~i] for i in range(len(s)))
all(is_palindrome(s.lower()) for s in palindromes)

True

## + and * operators

`+`와 `*`는 강력합니다. 간단하고 짧은 코드를 짤 수 있게 해주죠.

In [16]:
l = [1, 2, 3]
l += [4, 5]
l

[1, 2, 3, 4, 5]

In [17]:
for i in range(1, 6):
    print('*' * i)
for i in reversed(range(1, 5)):
    print('*' * i)

*
**
***
****
*****
****
***
**
*


2차원 배열도 쉽게 만들어줍니다.

In [18]:
row, col = 5, 3
board = [['-'] * col for _ in range(row)]
board

[['-', '-', '-'],
 ['-', '-', '-'],
 ['-', '-', '-'],
 ['-', '-', '-'],
 ['-', '-', '-']]

### A += Assgnment Puzzer: On Tuple!

이게 굉장히 재밌는 퍼즐입니다. 보통 tuple 이라면 *immutable* 이라고 생각하는데요. 아주 이상한 일이 발생합니다.

In [19]:
t = (1, 2, [3, 4])
print(t)
print(id(t))
print(id(t[2]))
# 이건 당연히 안됩니다
t[2] += [5, 6]
print(t)
print(id(t))
print(id(t[2]))

(1, 2, [3, 4])
4599818856
4599841736


TypeError: 'tuple' object does not support item assignment

In [20]:
t = (1, 2, [3, 4])
print(t)
print(id(t))
print(id(t[2]))
# 이건 됩니다... lol
t[2].append(5)
t[2].append(6)
print(t)
print(id(t))
print(id(t[2]))

(1, 2, [3, 4])
4599818928
4600016840
(1, 2, [3, 4, 5, 6])
4599818928
4600016840


굉장히 혼란스럽습니다. 설명충이 등장하자면요
- 첫예제는 t[2] 에 `+=` 연산자가 들어가며 t[2]의 id를 바꾸려하기에 실패합니다.
- 둘째는 t[2]의 내용물이 바뀔분, t[2]의 id가 바뀌지 않기에 모든게 잘됩니다... lol

## list.sort() and built-in functions

`list.sort()`는 in-place sort, `sorted()`는 아니라는 정도만 아시면 될 것 같습니다.

## Bisect

`bisect.bisect`, `bisect.bisect_left` 는 자주 쓰는 써치 api 들입니다. 간단하게 알아보겠습니다.

In [21]:
import bisect
l = list(range(0, 20, 2))
print(l)
print(bisect.bisect_left(l, 10))  # Locate the insertion point for x in a to maintain sorted order. 
print(bisect.bisect(l, 10)) # Similar to bisect_left(), but returns an insertion point which comes after (to the right of) any existing entries of x in a.

[0, 2, 4, 6, 8, 10, 12, 14, 16, 18]
5
6


## When `list` is not an answer

당연히 `list`, `tuple` 이 만능은 아닙니다. 예를들어, scientific computing 을 할때에는 데이터 양이 많아지니 compact 한 자료구조가 필요하겠죠?

그럴때 사용하는 것들이, `array.array`, `memoryview`, `numpy`, `scipy` 등이 있습니다.

`collections.deque`는 큐 관련 연산들을 잘 지원해줍니다. 큐 관련 정리를 해보면요.

| Collection | Desc |
|------------|------|
| collections.deque | thread-safe double-ended queue for fast inserting and removing from both ends |
| queue package | synchronized classes Queue, LifoQueue, PriorityQueue |
| multiprocessing package |  very similar to queue.Queue, but designed for interprocess communication |
| asyncio package | Queue, LifoQueue, PriorityQueue, JoinableQueue for asynchronous programming |
| heapq package | provides functions for priority queue |

### heapq
`heapq`는 힙, 혹은 프라이어리티 큐라는 이름으로 많이 사용됩니다. [leetcode](https://leetcode.com/problemset/all/) 에서도 많은 문제에 사용될 수 있구요.

예를들어, 이 문제를 보세요:

    In a stream, you are supposed to take the `k` longest strings seen so far. How do you do?
    
힙을 사용하면 쉽게 풀 수 있습니다.

In [22]:
import heapq
import itertools

def top_k(k, stream):
    min_heap = [(len(s), s) for s in itertools.islice(stream, k)]
    heapq.heapify(min_heap)
    for next_string in stream:
        heapq.heappushpop(min_heap, (len(next_string), next_string))
    return [p[1] for p in heapq.nsmallest(k, min_heap)]

top_k(2, iter(['the first longest string', 'a', 'b', 'ahahahah',
               'd', 'the second longest', 'the third']))

['the second longest', 'the first longest string']

## Tim Peters and Timsort

`list.sort()`나 `sorted()`에 사용되는 알고리즘은 [Timsort](https://en.wikipedia.org/wiki/Timsort)라고 합니다.

데이터의 상태에 따라서 insertion sort, merge sort 를 바꿔서 사용하는 알고리즘이라고 하는데요. Zen of Python을 쓴 위대한 Tim Peters 라는 분이 2002년에 처음으로 파이썬에서 사용하고, 자바, 안드로이드에서도 2009년에 사용했다고 합니다.